In [1]:
import pandas as pd

In [17]:
df = pd.read_html('https://fbref.com/es/equipos/db3b9613/Atletico-Madrid-Stats#all_stats_keeper', header=1)[2]
df

,Jugador,País,Posc,Edad,PJ,Titular,Mín,90 s,GC,GC90,...,PE,PP,PaC,PaC%,TPint,PD,PD.1,PC,% Salvadas.1,Partidos
0,Jan Oblak,si SVN,PO,32-133,36,36,3240,36.0,30,0.83,...,10,6,15,41.7,5,5,0,0,0.0,Partidos
1,Juan Musso,ar ARG,PO,31-014,1,1,90,1.0,0,0.00,...,0,0,1,100.0,0,0,0,0,NaN,Partidos
2,Total del equipo,NaN,NaN,29.2,37,37,3330,37.0,30,0.81,...,10,6,16,43.2,5,5,0,0,0.0,NaN
3,Total del oponente,NaN,NaN,27.6,37,37,3330,37.0,64,1.73,...,10,21,6,16.2,7,6,0,1,0.0,NaN


In [18]:
df = df.iloc[:-2]

print("\nDataFrame después de quitar las 2 últimas filas (modificado en la misma variable 'df'):")
print(df)


DataFrame después de quitar las 2 últimas filas (modificado en la misma variable 'df'):
      Jugador    País Posc    Edad  PJ  Titular   Mín  90 s  GC  GC90  ...  \
0   Jan Oblak  si SVN   PO  32-133  36       36  3240  36.0  30  0.83  ...   
1  Juan Musso  ar ARG   PO  31-014   1        1    90   1.0   0  0.00  ...   

   PE  PP  PaC   PaC%  TPint  PD  PD.1  PC  % Salvadas.1  Partidos  
0  10   6   15   41.7      5   5     0   0           0.0  Partidos  
1   0   0    1  100.0      0   0     0   0           NaN  Partidos  

[2 rows x 24 columns]


In [19]:

# --- INICIO DEL ANÁLISIS ---

# 1. Asegurar que las columnas clave son numéricas
# Es crucial que 'Mín' y '% Salvadas' sean numéricas para poder operar con ellas.
# A veces, pd.read_html puede cargar números como strings si hay caracteres no numéricos.
# Usaremos .replace(',', '.') para manejar comas como separadores decimales si es necesario
# y errors='coerce' para convertir cualquier valor no numérico a NaN.

df['Mín'] = pd.to_numeric(df['Mín'], errors='coerce')
# Observa que hay dos columnas '% Salvadas'. Supondré que la primera es la relevante.
df['% Salvadas'] = pd.to_numeric(df['% Salvadas'], errors='coerce')

# 2. Eliminar filas con valores nulos en las columnas clave para el análisis
# Esto es importante si errors='coerce' convirtió algunos valores a NaN.
df.dropna(subset=['Mín', '% Salvadas'], inplace=True)

# 3. Establecer un umbral mínimo de minutos jugados para un análisis significativo
MIN_MINUTES_THRESHOLD = 300

# 4. Filtrar a los porteros que han jugado al menos el umbral de minutos
porteros_calificados = df[df['Mín'] >= MIN_MINUTES_THRESHOLD]

if not porteros_calificados.empty:
    # 5. Ordenar por '% Salvadas' de forma descendente para encontrar el mejor
    mejor_portero = porteros_calificados.sort_values(by='% Salvadas', ascending=False).iloc[0]

    print("\n--- Análisis de Porteros Calificados ---")
    print(f"Umbral mínimo de minutos jugados: {MIN_MINUTES_THRESHOLD} minutos.")
    print("\nEl portero con el mejor porcentaje de salvadas (entre los calificados) es:")
    print(f"  Jugador: {mejor_portero['Jugador']}")
    print(f"  Minutos Jugados: {mejor_portero['Mín']:.0f}")
    print(f"  Porcentaje de Salvadas: {mejor_portero['% Salvadas']:.2f}%")
    print(f"  Goles Encajados por 90 min: {mejor_portero['GC90']:.2f}")

else:
    print(f"\nNo hay porteros que cumplan con el umbral de {MIN_MINUTES_THRESHOLD} minutos jugados.")


--- Análisis de Porteros Calificados ---
Umbral mínimo de minutos jugados: 300 minutos.

El portero con el mejor porcentaje de salvadas (entre los calificados) es:
  Jugador: Jan Oblak
  Minutos Jugados: 3240
  Porcentaje de Salvadas: 78.10%
  Goles Encajados por 90 min: 0.83


<ipython-input-19-f3b7c55ca49c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Mín'] = pd.to_numeric(df['Mín'], errors='coerce')
<ipython-input-19-f3b7c55ca49c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['% Salvadas'] = pd.to_numeric(df['% Salvadas'], errors='coerce')
<ipython-input-19-f3b7c55ca49c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index